# Week1_Library 과제

### Q1. Library 와 Framework 의 차이 간단하게 서술하시오. (100자 내외)

프레임워크와 라이브러리의 차이는 흐름에 대한 제어 권한이 어디에 있는가 이다. 프레임워크는 전체적인 흐름을 자체적으로 가지고 있으며, 프로그래머가 그 안에 필요한 코드를 작성하는 반면 라이브러리는 사용자가 흐름에 대한 제어를 하며 필요한 상황에 가져다 쓰는 것이다.

### Q2. 딥러닝과 머신러닝의 관계 및 특징, 차이 간단하게 서술하시오. (200자 내외)

머신러닝은 지도학습, 비지도학습, 강화학습을 통해 학습 방법을 스스로 개선하여 문제를 해결하는 인공지능의 한 분야이며 딥러닝은 인고ㅇ신경망을 통해 학습하고 문제를 해결하는 머신러닝의 하위 개념이다. 머신러닝은 구조화된 데이터를 필요로하고 전통적인 알고리즘을 사용하는 반면 딥러닝은 신경망을 통해 대량의 비정형 데이터를 수용하도록 구축된다.

### Q3. 아래의 코드에 주석 달기.

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision
import torchvision.transforms as transfroms

# cuda 쓸 수 있으면 쓰고 아니면 cpu 씀
device = 'cuda' if torch.cuda.is_available() else 'cpu'
torch.manual_seed(45) # 시드고정
if device == 'cuda':
    torch.cuda.manual_seed_all(45)
print(device + " is available")

In [ ]:
# 하이퍼파라미터 설정
learning_rate = 0.001
batch_size = 100
num_classes = 10
epochs = 5

In [ ]:
# 데이터 불러오기

train_set = torchvision.datasets.MNIST( # MNIST 데이터 셋 불러옴
    root = './data/MNIST', # 데이터 다운 경로
    train = True, # training data 리턴
    download = True, # 경로에 데이터 없을경우 다운로드 받겠다
    transform = transfroms.Compose([
        transfroms.ToTensor() # 텐서로 변환
    ])
)

test_set = torchvision.datasets.MNIST(# MNIST 데이터 셋 불러옴
    root = './data/MNIST',# 데이터 다운 경로
    train = False,# test data 리턴
    download = True, # 경로에 데이터 없을경우 다운로드 받겠다
    transform = transfroms.Compose([
        transfroms.ToTensor()  # 텐서로 변환
    ])
)

In [ ]:
# 불러온 mnist 데이터 dataloader로 데이터 로드함
train_loader = torch.utils.data.DataLoader(train_set, batch_size=batch_size)
test_loader = torch.utils.data.DataLoader(test_set, batch_size=batch_size)

# 데이터 쉐입확인
examples = enumerate(train_set)
batch_idx, (example_data, example_targets) = next(examples)
example_data.shape # 출력시 ** torch.Size([1, 28, 28]) **

In [ ]:
class ConvNet(nn.Module):
  def __init__(self): 
        super(ConvNet, self).__init__()
        
        # convolution layer 1 : input 텐서의 채널 수 = 1, ouput 텐서의 채널 수 = 10, 커널 사이즈 = 5
        self.conv1 = nn.Conv2d(1, 10, kernel_size=5) 
        # convolution layer 2 : input 텐서의 채널 수 = 10, ouput 텐서의 채널 수 = 20, 커널 사이즈 = 5
        self.conv2 = nn.Conv2d(10, 20, kernel_size=5) 
        # dropout 층 : dropout 비율 0.25
        self.drop2D = nn.Dropout2d(p=0.25, inplace=False) 
        # maxpooling 층 : 커널사이즈 = 2, 스트라이드 = 2
        self.mp = nn.MaxPool2d(2)
        # fully-Connected layer 1 : input_dim = 320, output_dim = 100 >> 100개 차원으로
        self.fc1 = nn.Linear(320,100) 
        # fully-Connected layer 2 : input_dim = 100, output_dim= 10 >> 10개 출력
        self.fc2 = nn.Linear(100,10) 

  def forward(self, x):
        x = F.relu(self.mp(self.conv1(x))) # convolution layer 1에 relu를 씌우고 maxpool
        x = F.relu(self.mp(self.conv2(x))) # convolution layer 2에 relu를 씌우고 maxpool
        x = self.drop2D(x) 
        x = x.view(x.size(0), -1) # Flatten : 첫번째 차원인 배치 차원은 그대로 두고 나머지는 펼침
        x = self.fc1(x) # fc1 레이어에 삽입
        x = self.fc2(x) # fc2 레이어에 삽입
        return F.log_softmax(x) # logsoftmax 적용

In [ ]:
model = ConvNet().to(device) # 모델생성
criterion = nn.CrossEntropyLoss().to(device) # 손실함수로 크로스엔트로피 사용
optimizer = torch.optim.Adam(model.parameters(), lr = learning_rate) # 최적화함수로 Adam 사용

In [ ]:
for epoch in range(epochs): 
    avg_cost = 0
    for data, target in train_loader:
        data = data.to(device)
        target = target.to(device)
        optimizer.zero_grad() # gradient 0으로 설정
        hypothesis = model(data) # 모델을 forward pass해 결과값 저장 
        cost = criterion(hypothesis, target) # output과 target의 loss 계산
        cost.backward() # gradient 계산
        optimizer.step() # 모델의 학습 파라미터 갱신
        avg_cost += cost / len(train_loader) # loss 값을 변수에 누적하고 train_loader의 개수로 나눔 = 평균
    print('[Epoch: {:>4}]  cost = {:>.9}'.format(epoch + 1, avg_cost))

In [ ]:
model.eval()# evaluate mode로 전환 dropout 이나 batch_normalization 해제
with torch.no_grad():# grad 해제  
    correct = 0
    total = 0

    for data, target in test_loader:
        data = data.to(device)
        target = target.to(device)
        out = model(data)
        preds = torch.max(out.data, 1)[1] # 출력이 분류 각각에 대한 값으로 나타나기 때문에, 가장 높은 값을 갖는 인덱스를 추출
        total += len(target) 
        correct += (preds==target).sum().item() # 예측값과 실제값이 같은지 비교
        
    print('Test Accuracy: ', 100.*correct/total, '%')
     

## 첫 정규세션 들으시느라 고생 많으셨습니다.